In [0]:
%run ./01-config

In [0]:
class HistoryLoader():
    def __init__(self,env):
        conf= Config()
        self.landing_zone= conf.base_dir_data + '/raw'
        self.testing_data_dir= conf.base_dir_data + '/testing'
        self.catalog=env
        self.db_name = conf.db_name
    def load_date_lookup(self):
        print(f'loading the date_lookup_table..',end ='')
        spark.sql(f""" insert overwrite table {self.catalog}.{self.db_name}.date_lookup 
                  select date,week,year,month,dayofweek,dayomonth,dayofyear,week_part
                   from json.`{self.testing_data_dir}/6-date-lookup.json/`
                  """)
        print('done')
    def load_history(self):
        import time
        start = int(time.time())
        print(f'loading the history table...',end='')
        self.load_date_lookup()
        print(f'history data load complete {int(time.time()-start)}seconds')
    def assert_count(self, table_name, expected_count):
        print(f'validating the record count..',end ='')
        actual_count =spark.read.table(f'{self.catalog}.{self.db_name}.{table_name}').count()
        assert actual_count == expected_count, f'expaected {expected_count,:} records, found {actual_count:,} in {table_name}'
        print('done')
    def validate(self):
        import time
        start = int(time.time())
        print(f'validating the data..',end ='')
        self.assert_count('date_lookup', 365)
        self.assert_count('history', 1000000)
        print(f'validation complete {int(time.time()-start)}seconds')   
